# Prérequis pour faire fonctionner le modéle MASK RCNN

In [ ]:

#Pour fonctionner correctement le framework Mask RCNN a besoin de la version keras==2.2.4 et tensorflow==1.14.0
!pip uninstall keras -y
!pip install keras==2.2.4
#!pip uninstall tensorflow -y
#!pip install tensorflow==1.14.0

In [1]:
#Verification des versions de keras et tensorflow

import tensorflow
import keras
import skimage

print(tensorflow.__version__)
print(keras.__version__)
print(skimage.__version__)

# C'est parti pour le modéle Mask RCNN!!

In [2]:

#Importation des modules utiles

import os
import gc
import sys
import time
import json
import glob
import random
from pathlib import Path
import pandas as pd

from PIL import Image
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa


In [3]:
#Chargement du DataFrame et nettoyage

train_df = pd.read_csv("../input/understanding_cloud_organization/train.csv")
train_df = train_df.dropna()
train_df.head()

# Restructuration du Dataframe

In [4]:
#les cellules suivantes permettent de restructurer le dataframe afin qu'il puisse alimenter le modéle  MaskRCNN. Pour chaque image , on a la liste
#des masques au format RLE ("EncodedPixels" colonne) et la catégorie du nuage ("CategoryId" colonne)

category_list = ["Fish","Flower","Gravel","Sugar"]

train_dict = {}
train_class_dict = {}
for idx, row in train_df.iterrows():
    image_filename = row.Image_Label.split("_")[0]
    class_name = row.Image_Label.split("_")[1]
    class_id = category_list.index(class_name)
    if train_dict.get(image_filename):
        train_dict[image_filename].append(row.EncodedPixels)
        train_class_dict[image_filename].append(class_id)
    else:
        train_dict[image_filename] = [row.EncodedPixels]
        train_class_dict[image_filename] = [class_id]

In [5]:
df = pd.DataFrame(columns=["image_id","EncodedPixels","CategoryId","Width","Height"])
for key, value in train_dict.items():
    img = Image.open("../input/understanding_cloud_organization/train_images/{}".format(key))
    width, height = img.width, img.height
    df = df.append({"image_id": key, "EncodedPixels": value, "CategoryId": train_class_dict[key], "Width": width, "Height": height},ignore_index=True)

In [6]:
df.head()

# Récuperation du modéle MaskRCNN et initialisation

In [7]:
#Parametres généraux 

DATA_DIR = Path('../kaggle/input/')
ROOT_DIR = "../working"

NUM_CATS = len(category_list) # Nombre de catégorie de nuages
IMAGE_SIZE = 512 # taille des images sur lesquelles nous allons travailler 512*512 au lieu de 2100 * 1400

In [8]:
#Chargement du framework modéle Mask RCNN depuis  git
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')

!rm -rf .git
!rm -rf images assets

In [40]:
#Import des fonctions utiles du Framework Mask RCNN

sys.path.append(ROOT_DIR+'/Mask_RCNN')
from mrcnn.config import Config

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

#Fonction pour resizer les images  :

def resize_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA)  
    return img

def refine_masks(masks, rois):
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    mask_index = np.argsort(areas)
    union_mask = np.zeros(masks.shape[:-1], dtype=bool)
    for m in mask_index:
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
        union_mask = np.logical_or(masks[:, :, m], union_mask)
    for m in range(masks.shape[-1]):
        mask_pos = np.where(masks[:, :, m]==True)
        if np.any(mask_pos):
            y1, x1 = np.min(mask_pos, axis=1)
            y2, x2 = np.max(mask_pos, axis=1)
            rois[m, :] = [y1, x1, y2, x2]
    return masks, rois


#Fonction utile pour calculer le DICE sur chaque image testée :

def dice_coef(y_true, y_pred, smooth=1):
    
    if y_true.shape[-1] == 0 or y_pred.shape[-1] == 0:
        return 0
    # flatten masks and compute their areas
    y_true_f = np.reshape(y_true > .5, (-1, y_true.shape[-1])).astype(np.float32)
    y_pred_f = np.reshape(y_pred  > .5, (-1, y_pred.shape[-1])).astype(np.float32)
    area1 = np.sum(y_true_f, axis=0)
    area2 = np.sum(y_pred_f, axis=0)
    
    intersections = np.dot(y_true_f.T, y_pred_f)
    
    dice= (2. * intersections + smooth) / (area1[:, None]+area2[None, :]+ smooth)
   
    
    return np.mean(dice.max(axis=1))

In [41]:
#Récupération des poids COCO pré-entrainés sur le modéle Mask RCNN
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = 'mask_rcnn_coco.h5'

In [52]:
#Fichier de Configuration du modéle Mask RCNN qui sera utilisé pour le mode Training et Inférence

class CloudConfig(Config):
    NAME = "cloud"
    NUM_CLASSES = NUM_CATS + 1 # nb de type de nuage + background
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 
    
    BACKBONE = 'resnet50'
    
    IMAGE_MIN_DIM = IMAGE_SIZE
    IMAGE_MAX_DIM = IMAGE_SIZE    
    IMAGE_RESIZE_MODE = 'none'
    
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    
    STEPS_PER_EPOCH = int(0.8*len(df))
    VALIDATION_STEPS = int(0.2*len(df))
    RPN_NMS_THRESHOLD=0.6
    DETECTION_MAX_INSTANCES=4
    
config = CloudConfig()
config.display()

# Péparation du Dataset pour le Mask RCNN contenant les images , masques et labels

In [42]:

#Utilisation de la class utils.Dataset du framework pour creer notre dataset de training er validation
#on surcharge les fonctions load_imag, load_mask qui viendront récuperer les images et les masks+labels des images

class CloudDataset(utils.Dataset):

    def __init__(self, df):
        super().__init__(self)
        
        # Add classes
        for i, name in enumerate(category_list):
            self.add_class("cloud", i+1, name)
        
        # Add images 
        for i, row in df.iterrows():
            self.add_image("cloud", 
                           image_id=row.name, 
                           path='../../input/understanding_cloud_organization/train_images/'+str(row.image_id), 
                           labels=row['CategoryId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [category_list[int(x)] for x in info['labels']]
    
    def load_image(self, image_id):
        return resize_image(self.image_info[image_id]['path'])

    def load_mask(self, image_id):
        info = self.image_info[image_id]
                
        mask = np.zeros((IMAGE_SIZE, IMAGE_SIZE, len(info['annotations'])), dtype=np.uint8)
        labels = []
        
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(info['height']*info['width'], 0, dtype=np.uint8)
            annotation = [int(x) for x in annotation.split(' ')]
            
            for i, start_pixel in enumerate(annotation[::2]):
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1

            sub_mask = sub_mask.reshape((info['height'], info['width']), order='F')
            sub_mask = cv2.resize(sub_mask, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
            
            mask[:, :, m] = sub_mask
            labels.append(int(label)+1)
            
        return mask, np.array(labels)

# Séparation des données d'entrainement et de validation (par ID unique)

In [30]:
#Creation du Dataset de Training  et de Validation. Split 80% /20%

training_percentage = 0.8

training_set_size = int(training_percentage*len(df))
validation_set_size = int((1-training_percentage)*len(df))
#training_set_size = 100
#validation_set_size = 1000


train_dataset = CloudDataset(df[:training_set_size])
train_dataset.prepare()

valid_dataset = CloudDataset(df[training_set_size:training_set_size+validation_set_size])
valid_dataset.prepare()

#Affichage de 5 images du dataset de training avec les masks

for i in range(5):
    image_id = i
    print(train_dataset.image_reference(image_id))
    
    image = train_dataset.load_image(image_id)
    mask, class_ids = train_dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, train_dataset.class_names, limit=4)

# Entrainement du modéle MASK RCNN

# Callbacks

In [55]:
#Création d'un callback MeanAverageDice qui ira calculer à la fin de chaque epcohs , le dice moyen  sur les données de validation

import keras
from mrcnn.model import *
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau 



class MeanAverageDice(Callback):
    def __init__(self, train_model: MaskRCNN, inference_model: MaskRCNN, dataset: CloudDataset,
                 calculate_dice_at_every_X_epoch=1, dataset_limit=99,
                 verbose=1):
        super().__init__()
        self.train_model = train_model
        self.inference_model = inference_model
        self.dataset = dataset
        self.calculate_dice_at_every_X_epoch = calculate_dice_at_every_X_epoch
        self.dataset_limit = len(self.dataset.image_ids)
        if dataset_limit is not None:
            self.dataset_limit = dataset_limit
        self.dataset_image_ids = self.dataset.image_ids.copy()

        if inference_model.config.BATCH_SIZE != 1:
            raise ValueError("This callback only works with the bacth size of 1")

        self._verbose_print = print if verbose > 0 else lambda *a, **k: None

    def on_epoch_end(self, epoch, logs=None):

        self._verbose_print("Dice moyen sur détection")
        self._load_weights_for_model()

        dices = self._calculate_mean_average_dice()
        dice_mean = np.mean(dices)

        logs["val_mean_dice"] = dice_mean

        self._verbose_print("Dice moyen sur detection a l' epoch {0} is: {1}".format(epoch+1,dice_mean))

        super().on_epoch_end(epoch, logs)

    def _load_weights_for_model(self):
        last_weights_path = self.train_model.find_last()
        self._verbose_print("Loaded weights for the inference model (last checkpoint of the train model): {0}".format(
            last_weights_path))
        self.inference_model.load_weights(last_weights_path,
                                          by_name=True)

    def _calculate_mean_average_dice(self):
        mAPs = []

        # random subset sur les données
        np.random.shuffle(self.dataset_image_ids)

        for image_id in self.dataset_image_ids[:self.dataset_limit]:
            image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(self.dataset, self.inference_model.config,
                                                                             image_id, use_mini_mask=False)
            molded_images = np.expand_dims(mold_image(image, self.inference_model.config), 0)
            results = self.inference_model.detect(molded_images, verbose=0)
            r = results[0]
            # Compute dice - VOC uses IoU 0.5
            dice_moyen=dice_coef(gt_mask.astype(np.float32),r['masks'].astype(np.float32))

            mAPs.append(dice_moyen)
                

        return np.array(mAPs)



#initialisation du callback MeanAverageDice
mean_average_dice_callback = MeanAverageDice(model,
model_inference, valid_dataset, calculate_dice_at_every_X_epoch=1, verbose=1)

In [56]:
#Initialisation du callback  ReduceLROnPlateau
reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         mode='min',
                                         episilon = 0.01,
                                         patience=3,
                                         factor=0.1,
                                         min_lr=1e-6,
                                         verbose=1)

# TRAINING

In [45]:
#Création de 2 instances  Mask RCNN , l'une pour l'entrainement ( training mode) , l'autre pour la détéction (inférence mode)

model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

model_inference = modellib.MaskRCNN(mode='inference', config=config, model_dir=ROOT_DIR)


In [47]:
#Chargement des poids


try:
    #initialisation du modéle pré-entrainé si existant
    model.load_weights('../../input/mask-rcnn-v2/mask_rcnn_cloud_0005-2.h5', by_name=True)
    print("mask_rcnn_cloud_0005-2.h5 loaded")
except:
    
    #initialisation du modéle d'entrainement avec les poids COCO
    model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    'mrcnn_class_logits', 'mrcnn_bbox_fc', 'mrcnn_bbox', 'mrcnn_mask'])
    print("mask rcnn coco loaded")
    pass

In [50]:
#Paramétres pour l'entrainement :Learning rate , nb d'epochs et augmentation lors de la génération d'image

LR = 1e-4
EPOCHS = 10


# Pour appliquer de l'augmentation aux images et masks lors de la génération des batchs

augmentation = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5)
], random_order=True)

In [ ]:
#Entrainement du modéle Mask RCNN sur le Dataset d'entrainement avec augmentation=augmentation et custom_callbacks=[mean_average_dice_callback,reduce_learning_rate]
#La taille des batchs est de 1
#On entraine le modéle sur toutes les couches layers='all'

model.train(train_dataset, valid_dataset,
            learning_rate=LR,
            epochs=EPOCHS,
            layers='all',
            augmentation=augmentation,custom_callbacks=[mean_average_dice_callback,reduce_learning_rate])

history = model.keras_model.history.history

# Evaluation

In [ ]:
#Affichage des graphiques

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='right')
plt.subplot(122)
plt.plot(history['val_mean_dice'])
plt.title('Model val_mean_dice by epoch')
plt.ylabel('val_mean_dice')
plt.xlabel('epoch')
plt.legend(['valid'], loc='right')



# Chargement des poids entrainés et visualisation

In [18]:
#Création d'une instance Mask RCNN en mode inférence

class InferenceConfig(CloudConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.6
    DETECTION_NMS_THRESHOLD = 0.3
    RPN_ANCHOR_SCALES = (16,64, 96, 128, 256)
    DETECTION_MAX_INSTANCES=4
    

inference_config = InferenceConfig()

model_detect = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=ROOT_DIR)

In [19]:
#Chargement des poids pré-entrainés

model_detect.load_weights('../../input/mask-rcnn-v2/mask_rcnn_cloud_0005-2.h5', by_name=True)

In [53]:
#Affichage des prédictions et masks originaux +dice par image testée sur 20 images du jeux de validation


for i in range(1,20, 1):
    image_id = df["image_id"][i+training_set_size]
    image_path = str('../../input/understanding_cloud_organization/train_images/'+image_id)
    print(image_path)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    result = model_detect.detect([resize_image(image_path)])
    r = result[0]
    
    if r['masks'].size > 0:
        masks = np.zeros((img.shape[0], img.shape[1], r['masks'].shape[-1]), dtype=np.uint8)
        for m in range(r['masks'].shape[-1]):
            masks[:, :, m] = cv2.resize(r['masks'][:, :, m].astype('uint8'), 
                                        (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        y_scale = img.shape[0]/IMAGE_SIZE
        x_scale = img.shape[1]/IMAGE_SIZE
        rois = (r['rois'] * [y_scale, x_scale, y_scale, x_scale]).astype(int)
        
        masks, rois = refine_masks(masks, rois)
    else:
        masks, rois = r['masks'], r['rois']
        
    visualize.display_instances(img, rois, masks, r['class_ids'], 
                                ['bg']+category_list, r['scores'],
                                title=image_id, figsize=(12, 12))
    print(valid_dataset.image_reference(i))
    image = valid_dataset.load_image(i)
    mask, class_ids = valid_dataset.load_mask(i)
    visualize.display_top_masks(image, mask, class_ids, valid_dataset.class_names, limit=5)
    
    #Affichage  du dice
    dice=dice_coef(mask.astype(np.float32),r['masks'].astype(np.float32))
    print("dice:",dice)

    
    

In [36]:
#calcul du dice moyen sur 500 images du jeux de validation

dice_moyen=[]

for i in range(1,500, 1):
    image_id = df["image_id"][i+training_set_size]
    image_path = str('../../input/understanding_cloud_organization/train_images/'+image_id)
    #print(image_path)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    result = model_detect.detect([resize_image(image_path)])
    r = result[0]
    
    if r['masks'].size > 0:
        masks = np.zeros((img.shape[0], img.shape[1], r['masks'].shape[-1]), dtype=np.uint8)
        for m in range(r['masks'].shape[-1]):
            masks[:, :, m] = cv2.resize(r['masks'][:, :, m].astype('uint8'), 
                                        (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        y_scale = img.shape[0]/IMAGE_SIZE
        x_scale = img.shape[1]/IMAGE_SIZE
        rois = (r['rois'] * [y_scale, x_scale, y_scale, x_scale]).astype(int)
        
        masks, rois = refine_masks(masks, rois)
    else:
        masks, rois = r['masks'], r['rois']
        

    #print(valid_dataset.image_reference(i))
    image = valid_dataset.load_image(i)
    mask, class_ids = valid_dataset.load_mask(i)
    
    #Affichage  du dice
    dice=dice_coef(mask.astype(np.float32),r['masks'].astype(np.float32))
    #print("dice:",dice)
    if dice>0:
        dice_moyen.append(dice)

print("Dice moyen sur les images de validation-sur détéction uniquement -:",np.mean(dice_moyen))

    